## 2.  핫플지수 확정



In [1]:
import polars as pl
import numpy as np

print("✅ 1️⃣ Parquet 스캔 시작")
lazy_df = pl.scan_parquet('combined_parquet/**/*.parquet', glob=True)
print("   → LazyFrame 생성 완료")


✅ 1️⃣ Parquet 스캔 시작
   → LazyFrame 생성 완료


In [8]:
import polars as pl

# ———————— 풀 테이블 출력 설정 ————————
pl.Config.set_tbl_rows(100)         # 최대 100행까지 줄임표 없이 표시
pl.Config.set_tbl_cols(100)        # 최대 100열까지 표시
pl.Config.set_tbl_formatting("UTF8_FULL")  # 테두리 포함, 생략 없이 전체 보여주기


polars.config.Config

In [2]:

import polars as pl


# 1️⃣ filtered 정의: 2024년··주요 목적 필터 + 날짜 파싱
filtered = (
    lazy_df
      # ─ 날짜 범위: 2024-01-01 ~ 2024-12-31
      .filter((pl.col("ETL_YMD") >= "20240101") & (pl.col("ETL_YMD") <= "20241231"))
      
    #   # ─ 시간 범위: 18시(18)부터 23시(23)까지
    #   .filter((pl.col("TIME_CD").cast(pl.Int64) >= 18) 
    #         & (pl.col("TIME_CD").cast(pl.Int64) <= 23))
      
    #   # ─ 목적 필터: 관심 목적군만
    #   .filter(pl.col("MOVE_PURPOSE").is_in(
    #       ["쇼핑","관광","출근","귀가","병원","기타","등교"]
    #   ))


      # ─ ETL_YMD 문자열을 Date 타입으로 바꿔서
      .with_columns(
          pl.col("ETL_YMD")
            .str.strptime(pl.Date, format="%Y%m%d")
            .alias("ETL_DATE")
      )

#       # ─ 평일(월~금)만 남기기 (weekday 0~4)
#       .filter(pl.col("ETL_DATE").dt.weekday().is_in([0,1,2,3,4]))
      
)

# 이제 `filtered`에 “원하는 기간·시간·목적·요일” 이 모두 적용되어 있으니,
# 아래처럼 곧장 동별 연간 통계 계산에 쓰실 수 있습니다.



In [ ]:
# 1️⃣ HOTNESS1 계산 & 상위30 평균


young_cols = ['MALE_20_CNT','MALE_30_CNT','FEML_20_CNT','FEML_30_CNT']
all_cols = young_cols + [ 
    'MALE_00_CNT','MALE_10_CNT','MALE_40_CNT','MALE_50_CNT','MALE_60_CNT','MALE_70_CNT',
    'FEML_00_CNT','FEML_10_CNT','FEML_40_CNT','FEML_50_CNT','FEML_60_CNT','FEML_70_CNT'
]

import polars as pl

# (1) 20·30대 합계와 전체 합계 Expr 정의
young_sum: pl.Expr = sum(
    (pl.col(c).cast(pl.Float64) for c in young_cols),
    pl.lit(0.0)
)         # “한 행(row)에서” 20·30대 합계

total_sum: pl.Expr = sum(
    (pl.col(c).cast(pl.Float64) for c in all_cols),
    pl.lit(0.0)
)         # “한 행(row)에서” 전체 방문자 합계


#  날짜·시간·목적 필터  
step1 = (
    lazy_df
      .filter((pl.col('ETL_YMD') >= '20240101') & (pl.col('ETL_YMD') <= '20241231'))
    #   .filter((pl.col('TIME_CD').cast(pl.Int64) >= 10) & (pl.col('TIME_CD').cast(pl.Int64) <= 23))
      .filter(pl.col('MOVE_PURPOSE').is_in(['쇼핑','관광','출근','귀가','병원','기타']))
)


step2 = (
    step1
      .with_columns(pl.col('ETL_YMD').str.strptime(pl.Date, format='%Y%m%d').alias('ETL_DATE'))
    #   .filter(pl.col('ETL_DATE').dt.weekday().is_in([5, 6]))
)

# 4-1️⃣ 먼저 YOUNG_VISITORS, TOTAL_VISITORS, PURPOSE_FLAG, YOUNG_RATIO 만 추가
step3 = step2.with_columns([
    young_sum.alias("YOUNG_VISITORS"),  #2030 합계
    total_sum.alias("TOTAL_VISITORS"),  #전체 합꼐

    (young_sum / (total_sum + 1e-6)).alias("YOUNG_RATIO"), #전체 중 2030 비율
])

print("\n▶ step3 샘플:")
print(step3
  .select(["YOUNG_RATIO", "FULL_NM","YOUNG_VISITORS","TOTAL_VISITORS", "ETL_DATE"])
  .limit(50)
  .collect()
)



▶ step3 샘플:
shape: (50, 4)
┌─────────────┬──────────────────────────────┬────────────────┬────────────────┐
│ YOUNG_RATIO ┆ FULL_NM                      ┆ YOUNG_VISITORS ┆ TOTAL_VISITORS │
│ ---         ┆ ---                          ┆ ---            ┆ ---            │
│ f64         ┆ str                          ┆ f64            ┆ f64            │
╞═════════════╪══════════════════════════════╪════════════════╪════════════════╡
│ 0.224515    ┆ 서울특별시_중구_신당5동      ┆ 27.42          ┆ 122.13         │
│ 0.414714    ┆ 서울특별시_동작구_상도3동    ┆ 12.74          ┆ 30.72          │
│ 0.153586    ┆ 경기도_하남시_덕풍1동        ┆ 9.08           ┆ 59.12          │
│ 0.306814    ┆ 서울특별시_양천구_목1동      ┆ 762.54         ┆ 2485.35        │
│ 0.491442    ┆ 경기도_안산시_상록구_사동    ┆ 172.84         ┆ 351.7          │
│ …           ┆ …                            ┆ …              ┆ …              │
│ 0.445463    ┆ 경기도_수원시_권선구_세류2동 ┆ 95.73          ┆ 214.9          │
│ 0.135808    ┆ 경기도_안성시_일죽면         ┆ 17.78          ┆ 130.92  

In [7]:
# 1️⃣ 동별 연간 합산 & 비율 계산
annual = (
    step3
      .group_by("FULL_NM")
      .agg([
        pl.sum("YOUNG_VISITORS").alias("ANNUAL_YOUNG"),
        pl.sum("TOTAL_VISITORS").alias("ANNUAL_TOTAL"),
      ])
      .with_columns([
        (pl.col("ANNUAL_YOUNG") / (pl.col("ANNUAL_TOTAL") + 1e-6))
          .alias("ANNUAL_YOUNG_RATIO")
      ])
      .collect()  # 이제 동 단위 수백 개 행만 메모리에 로드
)

# 2️⃣ ANNUAL_YOUNG_RATIO로 상위 30개 추출
top30 = (
    annual
      .sort("ANNUAL_YOUNG_RATIO", descending=True)
      .head(30)    # .head(30)도 좋고 .limit(30).collect() 여기도 엑션이지만 이미 eager
)

print("▶ 동별 연간 YOUNG_RATIO TOP30:")
print(top30)

# 3️⃣ 상위30 평균·중앙값 계산
mean_30   = top30["ANNUAL_YOUNG_RATIO"].mean()
median_30 = top30["ANNUAL_YOUNG_RATIO"].median()

print(f"\n▶ TOP30 평균 : {mean_30:.6f}")
print(f"▶ TOP30 중앙값 : {median_30:.6f}")

# 4️⃣ 난향동 연간 YOUNG_RATIO
nanhyang = annual.filter(pl.col("FULL_NM") == "서울특별시_관악구_난향동")
print("\n▶ 난향동 연간 YOUNG_RATIO:")
print(nanhyang.select("ANNUAL_YOUNG_RATIO"))


▶ 동별 연간 YOUNG_RATIO TOP30:
shape: (30, 4)
┌─────────────────────────────┬──────────────┬──────────────┬────────────────────┐
│ FULL_NM                     ┆ ANNUAL_YOUNG ┆ ANNUAL_TOTAL ┆ ANNUAL_YOUNG_RATIO │
│ ---                         ┆ ---          ┆ ---          ┆ ---                │
│ str                         ┆ f64          ┆ f64          ┆ f64                │
╞═════════════════════════════╪══════════════╪══════════════╪════════════════════╡
│ 서울특별시_마포구_서교동    ┆ 3.8086e7     ┆ 6.3691e7     ┆ 0.597979           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_마포구_연남동    ┆ 6.7546e6     ┆ 1.1672e7     ┆ 0.578721           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_광진구_화양동    ┆ 1.2942e7     ┆ 2.2816e7     ┆ 0.567239           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 서울특별시_관악구_신림동    ┆ 7.7443e6     ┆ 1.4307e7     ┆ 0.541288           │
├╌╌╌╌╌

polars.config.Config